<a href="https://colab.research.google.com/github/ShivamSinghal1/FacialEmotionRecognition/blob/master/Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import time

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

### Importing Dataset

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Dataset/fer2013.csv')

In [ ]:
df.head()

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
df.emotion.value_counts()

### Visualize Dataset

In [ ]:
fig = plt.figure(1, (14, 14))

k = 0
for label in sorted(df.emotion.unique()):
    for j in range(1):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px , cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_label_to_text[label])
        plt.tight_layout()

### Input and Output Data

In [ ]:
X = df.pixels.apply(lambda x: np.array(x.split(' ')).astype('float32'))
X = np.stack(X, axis=0)
Y = np.array(df['emotion'])

In [ ]:
X.shape

In [ ]:
Y.shape

# Preprocessing
Face Alignment 

In [ ]:
import cv2

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
def face_align(gray):
  sz = gray.shape
  # Creating variable eyes
  eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
  if len(eyes) > 1 :
    index=0
    # Creating for loop in order to divide one eye from another
    for (ex , ey,  ew,  eh) in eyes:
      if index == 0:
        eye_1 = (ex, ey, ew, eh)
      elif index == 1:
        eye_2 = (ex, ey, ew, eh)
      index += 1
    if eye_1[0] < eye_2[0]:
      left_eye = eye_1
      right_eye = eye_2
    else:
      left_eye = eye_2
      right_eye = eye_1
        
    # Calculating coordinates of a central points of the rectangles
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    left_eye_x = left_eye_center[0] 
    left_eye_y = left_eye_center[1]
        
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    right_eye_x = right_eye_center[0]
    right_eye_y = right_eye_center[1]

    if left_eye_y > right_eye_y:
      A = (right_eye_x, left_eye_y)
      # Integer -1 indicates that the image will rotate in the clockwise direction
      direction = -1 
    else:
      A = (left_eye_x, right_eye_y)
      # Integer 1 indicates that image will rotate in the counter clockwise direction
      direction = 1 
        
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi

    # Width and height of the image
    h, w = gray.shape[:2]
    # Calculating a center point of the image
    # Integer division "//"" ensures that we receive whole numbers
    center = (w // 2, h // 2)
    # Defining a matrix M and calling
    # cv2.getRotationMatrix2D method
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    # Applying the rotation to our image using the
    # cv2.warpAffine method
    gray = cv2.warpAffine(gray, M, (w, h))

    #Again detecting face
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0 : 
      (x,y,w,h) = faces[0]
      gray = gray[y:y+h, x:x+w]

  gray = cv2.resize(gray,sz) 
  return gray

In [ ]:
X_new = []
for j in range(X.shape[0]):
  px = np.array(X[j]).reshape(48,48)
  px = np.array(px, dtype='uint8')
  px = face_align(px)
  X_new.append(np.array(px).reshape(48*48))
X = np.array(X_new)

In [ ]:
px = cv2.imread('/content/drive/My Drive/Dataset/Robert-Downey-Jr-Avengers-Endgame.jpg')
plt.imshow(px)
gray=cv2.cvtColor(px, cv2.COLOR_BGR2GRAY)
gray = face_align(gray)
plt.imshow(gray)

### Confusion Matrix

In [ ]:
# Plot confusion matrix 
from sklearn.metrics import confusion_matrix


def create_confmat(true_labels, predicted_labels, columns, colour = 'Greens', size = (20,14)):
    
    cm = confusion_matrix(true_labels, predicted_labels) 
    cm_df = pd.DataFrame(cm,
    index = [col for col in columns], 
    columns = [col for col in columns])
    plt.figure(figsize=(18,16))
    sns.heatmap(cm_df, annot = True, cmap = colour, fmt='g', linewidths=.2)
    plt.title('Confusion Matrix', fontsize = 20)
    plt.ylabel('True label', fontsize = 18)
    plt.xlabel('Predicted label', fontsize = 18)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()

### Train and Test Split for KNN


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y,
                                                    shuffle=True, stratify=Y,
                                                    test_size=0.01, random_state=2)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

## SVM

In [ ]:
from sklearn.svm import SVC
model = SVC()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_valid , y_valid)

We have now X_train (input) and y_train (output) Data for Training and X_valid and y_valid for Testing.

# KNN

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
def distance(v1, v2):
	# Eucledian 
	return np.sqrt(((v1-v2)**2).sum())

In [ ]:
def knn(X_train, Y_train , test , num ):
	dist = []
	
	for i in range(X_train.shape[0]):
		# Compute the distance from test point
		d = distance(test, X_train[i])
		dist.append([d, Y_train[i]])
	# Sort based on distance 
	dk = sorted(dist, key=lambda x : x[0])
	# Retrieve only the labels
	labels = np.array(dk)[:,-1]
	output = np.zeros(7)
	answer = []
	j = 0
	for k in range(num):
		output[int(labels[k])]+=1
		#u,indices = np.unique(output,return_counts = True)
		#if indices[-1] == 1 or k == num - 1: 
		#	while j <= k : 
		answer.append( np.argmax(output) )
		#		j+=1
	return answer

	# Get frequencies of each label
	#output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	#index = np.argmax(output[1])
	#return output[0][index]

In [ ]:
def wknn(X_train, Y_train , test , num ):
	dist = []
	for i in range(X_train.shape[0]):
		# Compute the distance from test point
		d = distance(test, X_train[i])
		if d == 0:
			d = 0.00000000001 
		dist.append([1/(d*d), Y_train[i]])
	# Sort reverse based on distance and get top k
	dk = sorted(dist, key=lambda x : x[0] , reverse = True)
	output = np.zeros(7)
	answer = []
	for k in range(num):
		output[int(dk[k][1])] += dk[k][0] 
		answer.append( np.argmax(output) )
	return answer

	# Get frequencies of each label
	#output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	#index = np.argmax(output[1])
	#return output[0][index]

In [ ]:
start_time = time.time()

num = 10
cnt = np.zeros(num)
y_pred_manually = []

for i in range(X_valid.shape[0]):
  answer = knn(X_train , y_train , X_valid[i] ,num)
  y_pred_manually.append(answer[0])
  for j in range(num):
    cnt[j] += answer[j] == y_valid[i]

accuracy = cnt/X_valid.shape[0]

print("Maximum Accuracy :",max(accuracy)*100 ,"%")
print("Value of K :",np.argmax(accuracy)+1)
print("Time taken to execute : %s seconds " % (time.time() - start_time))

In [ ]:
plt.style.use('seaborn')

plt.plot(list(range(1,11)) , accuracy , marker= 'o')
plt.legend(loc = 'upper left',fontsize = 16)
plt.xlabel("Value of K",fontsize = 20)
plt.ylabel("Accuracy",fontsize = 20)
plt.show()

In [ ]:
start_time = time.time()

num = 10
cnt = np.zeros(num)
y_pred_manually = []

for i in range(X_valid.shape[0]):
  answer = wknn(X_train , y_train , X_valid[i] ,num)
  y_pred_manually.append(answer[0])
  for j in range(num):
    cnt[j] += answer[j] == y_valid[i]

accuracy = cnt/X_valid.shape[0]

print("Maximum Accuracy :",max(accuracy)*100 ,"%")
print("Value of K :",np.argmax(accuracy)+1)
print("Time taken to execute : %s seconds " % (time.time() - start_time))

In [ ]:
plt.style.use('seaborn')

plt.plot(list(range(1,11)) , accuracy , marker= 'o')
plt.legend(loc = 'upper left',fontsize = 16)
plt.xlabel("Value of K",fontsize = 20)
plt.ylabel("Accuracy",fontsize = 20)
plt.show()

Applying KNN using SkLearn Library

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

In [ ]:
start_time = time.time()
y_pred = classifier.predict(X_valid)
print("Time Taken : %s seconds " % (time.time() - start_time))

In [ ]:
cnt = 0
for i in range(X_valid.shape[0]):
   cnt += y_pred[i] == y_valid[i]
print("Accuracy :", cnt/X_valid.shape[0]*100, "%")

Accuracy with Sklearn Library function is exactly same as manually implemented code.

In [ ]:
# Checking manually code prediction with the library code prediction
cnt = 0
for i in range(X_valid.shape[0]):
   cnt += y_pred[i] == y_pred_manually[i]
print("Accuracy :", cnt/X_valid.shape[0]*100, "%")
# All Prediction on these examples are exactly same

END KNN

# Applying CNN 




In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y,
                                                    shuffle=True, stratify=Y,
                                                    test_size=0.1, random_state=2)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential , load_model
from keras.layers import Convolution2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten, AveragePooling2D
from keras.initializers import  RandomNormal
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, TensorBoard ,EarlyStopping, ReduceLROnPlateau

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.normalize(X_train)
X_valid = preprocessing.normalize(X_valid)

In [ ]:
X_train = X_train.reshape(X_train.shape[0],48,48,1)
X_valid = X_valid.reshape(X_valid.shape[0],48,48,1)

In [ ]:
from keras.utils import np_utils
num_classes = 7
print("Shape before one-hot encoding: ", y_train.shape)
y_train = np_utils.to_categorical(y_train, num_classes)
y_valid = np_utils.to_categorical(y_valid, num_classes)
print("Shape after one-hot encoding: ", y_train.shape)

print(X_train.shape , y_train.shape)
print(X_valid.shape , y_valid.shape)

In [ ]:
def get_cnn_model():
    model = Sequential()

    model.add(Convolution2D(64, (3, 1), padding='same', input_shape=(48,48,1)))
    model.add(Convolution2D(64, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, (3, 1), padding='same'))
    model.add(Convolution2D(128, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(256, (3, 1), padding='same'))
    model.add(Convolution2D(256, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Convolution2D(512, (3, 1), padding='same'))
    model.add(Convolution2D(512, (1, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(7))
    model.add(Activation('softmax'))
    return model

model = get_cnn_model()
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

# Saving model each time it achieves lower loss on the validation set
filepath='/content/drive/My Drive/Dataset/VGG16.hdf5'
checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=8,
                          verbose=1,
                          restore_best_weights=True, mode = 'auto'
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.9,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)


history = model.fit(X_train , y_train, batch_size = 32
                   , epochs=35, 
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpointer,earlystop,reduce_lr]
                             )

pd.DataFrame(history.history).to_csv("history.csv")

In [ ]:
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

In [ ]:
# Plot Confusion Matrix
columns = {'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'}
cm_df = pd.DataFrame(cm,
    index = [col for col in columns], 
    columns = [col for col in columns])

plt.figure(figsize=(18,16))
sns.heatmap(cm_df, annot = True, cmap = 'Greens', fmt='g', linewidths=.2)
plt.title('Confusion Matrix', fontsize = 20)
plt.ylabel('True label', fontsize = 18)
plt.xlabel('Predicted label', fontsize = 18)
plt.tick_params(axis='both', labelsize=14)
plt.show()


## END OF CNN 

In [ ]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()
  
def f1score(label , confusion_matrix):
    p = precision(label , confusion_matrix)
    r = recall(label , confusion_matrix)
    return 2*p*r/(p+r)

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

def f1score_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_f1score = 0
    for label in range(columns):
        print('label',label , ':', f1score(label, confusion_matrix))
        sum_of_f1score += f1score(label, confusion_matrix)
    return sum_of_f1score / columns

def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 

In [ ]:
accuracy(cm)

In [ ]:
print("precision total:", precision_macro_average(cm))

print("recall total:", recall_macro_average(cm))

In [ ]:
print("f1score total:", f1score_macro_average(cm))